In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Cargar de CSV

In [60]:
df = pd.read_csv('Data/US_Crime_Data.csv')

Diccionario para reemplazar datos de la columna de State

In [61]:
key = ['ak','al','ar','az','ca','co','ct','de','fl','ga','hi','ia','id','il','in','ks',
'ky','la','ma','md','me','mi','mn','mo','ms','mt','nc','nd','ne','nh','nj','nm','nv','ny',
'oh','ok','or','pa','ri','sc','sd','tn','tx','ut','va','vt','wa','wi','wv','wy']

value = ['Alaska', 'Alabama', 'Arkansas', 'Arizona', 'California', 'Colorado', 'Connecticut',
         'Delaware', 'Florida', 'Georgia', 'Hawai', 'Iowa', 'Idaho', 'Illinois',
         'Indiana', 'Kansas', 'Kentucky', 'Louisiana', 'Massachusetts', 'Maryland', 'Maine', 
         'Michigan', 'Minnesota', 'Missouri', 'Misisipi', 'Montana', 'North Carolina', 'North Dakota',
         'Nebraska', 'New Hampshire', 'New Jersey', 'New México', 'Nevada', 'New York', 'Ohio','Oklahoma',
         'Oregon', 'Pensilvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas',
         'Utah', 'Virginia', 'Vermont', 'Washington', 'Wisconsin', 'West Virginia', 'Wyoming']

dic = dict(zip(key, value))

Correcion el la columna de State  "Algunos de los stados no son estados son ciudades o ubicaciones"

In [62]:

filtro1 = (df['State'].str.lower() == 'DC'.lower() ) | (df['State'].str.lower() =='D.C.'.lower() ) | (df['State'].str.lower() =='District Of Columbia'.lower())
df[filtro1] = 'wa'

filtro2 = (df['State'].str.lower() =='Los Angeles, California'.lower()) | (df['State'].str.lower() =='Venice Beach, Ca 90291'.lower()) | (df['State'].str.lower() =="New York And California".lower()) 
df[filtro2] = 'ca'


filtro3 = (df['State'].str.lower() =='Chicago'.lower())
df[filtro3] = 'il'

filtro3 = (df['State'].str.lower() =='io'.lower())
df[filtro3] = 'ia'

Transformacion de la data

In [63]:
df['City'] = df['City'].str.capitalize()
df['State'] = df['State'].str.lower()
df['State']= df['State'].apply(lambda x : str(x).replace('.', ''))
""" Alexandria Va es una Ciudad de Virginia """
fil = df['State'] == 'alexandria va'
""" Se actualiza todos los State de alexandria va a Virgina """
df[fil] = 'Virginia'

In [64]:
""" Actualizacion de la columna State """
for i,j in enumerate(df['State']):
    if j in dic.keys():
        for k,m in enumerate(dic):
            if j == m:
                df['State'][i] = dic[m]
df['State'] = df['State'].str.title()

Funcion para obtener DataFrame de datos agrupados con columna de conteo

In [65]:
def ContarColum(df_count):
    df_count = pd.DataFrame({'Count': df_count}).reset_index()
    return df_count

Cantidad de crimenes por ciudad

In [66]:
cantidad_city = ContarColum(df.groupby(by='City').size())
""" Grafico de barras """
#plt.bar(cantidad_city['City'], cantidad_city['Count'])
#plt.show()

' Grafico de barras '

Cantidad de crimenes por Estado

In [67]:
cantidad_state = ContarColum(df.groupby(by=['State']).size())
""" Grafico de barras """
#plt.bar(cantidad_state['State'], cantidad_state['Count'])
#plt.show()


' Grafico de barras '

Tabla de Estados

In [77]:
""" Dim States """
df_state = df.groupby(by='State').mean(numeric_only=True).reset_index()
df_state.rename(columns={'State':'Name_State'},inplace=True)

""" Data Frame de Codes State """
df_codes_state = pd.DataFrame({'Code':key, 'State':value})

""" Merge para crear el Dim_States """
df_state = df_state.merge(df_codes_state, how = 'left',left_on='Name_State', right_on='State',)

""" Eliminar columna inecesaria del merge"""
df_state.pop('State')
df_state['Code_State'] = df_state['Code'].str.upper()
df_state.pop('Code')

0      al
1      ak
2      az
3      ar
4      ca
5      co
6      ct
7      de
8      fl
9      ga
10     hi
11     id
12     il
13     in
14     ia
15     ks
16     ky
17     la
18     me
19     md
20     ma
21     mi
22     mn
23     ms
24     mo
25     mt
26    NaN
27     ne
28     nv
29     nh
30     nj
31     nm
32     ny
33     nc
34     nd
35     oh
36     ok
37     or
38    NaN
39     pa
40     ri
41     sc
42    NaN
43     sd
44     tn
45     tx
46     ut
47     vt
48     va
49     wa
50     wv
51     wi
52     wy
Name: Code, dtype: object

Tabla de Ciudades Con estados

In [78]:
""" Dim City """
df_city = df.groupby(by='City').mean(numeric_only=True).reset_index()
df_city.rename(columns={'City':'Name_City'},inplace=True)

""" Merge para crear el Dim_States """
df2 = df.groupby(by=['City', 'State']).mean(numeric_only=True).reset_index()
df_city = df_city.merge(df2, how = 'left', left_on='Name_City', right_on='City',)

""" Agregando el id_code del la dim_states """
df_city = df_city.merge(df_state, how = 'left', left_on='State', right_on='Name_State')

""" Eliminar columna innecesaria del merge """
df_city.pop('City')
df_city.pop('State')
df_city


,Name_City,Name_State,Code_State
0,Aberdeen,South Dakota,SD
1,Aberdeen,Washington,WA
2,Abernathy,Texas,TX
3,Abilene,Texas,TX
4,Adelphi,Maryland,MD
...,...,...,...
904,York,Pensilvania,PA
905,Youngstown,Ohio,OH
906,Ypsilanti,Michigan,MI
907,Yuma,Arizona,AZ
